#### Step 1: Prepare documents

In [20]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

In [21]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

## Q1 Answer : 1000

#### Step 2: Create Embeddings using Pretrained Models

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Q2 Answer : [120, 59.76] obtained from this link https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview all-MiniLM-L12-v2 Model Size 120MB , Avg Performance 59.76  

In [10]:
len(model.encode("This is a simple sentence"))

384

## Q2 Answer : 384

In [22]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

In [23]:
# Establish connection to Elasticsearch
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '95fd7ce64e68', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'CPvsNOx8QDKoLc40I68BBg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [25]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [26]:
 ## Q4 : In the es_client.indices.delete statement, what does "ignore_unavailable" mean?
# Answer : If the index is unavailable, skip this statement

In [30]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"][:1]

[{'_index': 'course-questions',
  '_id': 'WX3sipABYtSGEyq6SUZa',
  '_score': 0.85204816,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}}]

## Q5Q5: Create end user query and perform semantic search
## [0.74, 'General course-related questions', 'data-engineering-zoomcamp']

In [53]:
knn_query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [56]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'WX3sipABYtSGEyq6SUZa',
  '_score': 3.4953294,
  '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'question_vector': [0.043121110647916794,
    0.05837032571434975,
    0.005022963974624872,
    0.028570540249347687,
    -0.022065451368689537,
    0.004731940571218729,
    -0.02545982040464878,
    -0.0008887201547622681,
    -0.16603532433509827,
    -0.004694771021604538,
    -0.05639374256134033,
    -0.1067831739783287,
    -0.06021316722035408,
    0.03971248120069504,
    -0.06361554563045502,
    0.03923672437667847,
    -0.0044054873287677765,
    -0.0083926385268569,
    0.1448369175195694,
    -0.07729384303092957,
    0.015076647512614727,
    0.02086198702454567,
    0.019407764077186584,
    -0.031069451943039894,
    0.0079320091754

#Q6: Perform Semantic Search & Filtering

Answer Do you see the results filtered only to "General course-related questions" section?
Yes